In [ ]:
from appworld import AppWorld, load_task_ids

task_ids = load_task_ids(dataset_name='dev')

world = AppWorld(task_id=task_ids[0], ground_truth_mode='full')

gt = world.task.ground_truth

gt.api_calls

In [ ]:
import json

trajectories = json.load(open("./evaluation_results/trajectories_[sample].json"))

task_ids = list(trajectories.keys())

trajectory = trajectories[task_ids[2]]

for msg in trajectory['trajectory']:
    if 'role' in msg and msg['role'] == 'user':
        print("================================== [ 👤 User ] ====================================")
        print(f"{msg['content']}")
    elif 'role' in msg and msg['role'] == 'assistant':
        print("================================== [ 🤖 Assistant ] ====================================")
        print(f"{msg['content']}")
    elif 'type' in msg and msg['type'] == 'function_call':
        print("================================== [ 🤖 Assistant ] ====================================")
        print(f"{json.loads(msg['arguments'])['code']}")
    elif 'type' in msg and msg['type'] == 'function_call_output':
        print("================================== [ 🌏 Environment Feedback ] ====================================")
        print(f"{msg['output']}")
    
    print("=====================================================================================\n\n")

================================== [ 👤 User ] ====================================
Using these 'APIs' and 'reflection history' of your previous actions, now generate code to solve the actual task:

My name is Edwin Wilson. 
My personal email is ed_wilson@gmail.com and phone number is 8506051563.

**Task**:
Give me a comma-separated list of top 3 most played indie song titles from across my Spotify song, album and playlist libraries.

**Reflection History**:
<reflection>
The Actor Agent failed to retrieve the correct top 3 most played indie song titles due to several issues: it did not utilize the required APIs such as `spotify.show_album`, `spotify.show_playlist`, and `spotify.show_song` to gather detailed song information, including play counts and genres. Additionally, the Actor Agent did not implement the logic to filter songs by genre and play count, as outlined in the ground truth code. To improve, the Actor Agent should ensure it uses the correct APIs and follows the logic for fi

In [ ]:
import json

def show_trajectory(file_path, task_idx):
    
    trajectories = json.load(open(file_path))

    task_ids = list(trajectories.keys())
    
    trajectory = trajectories[task_ids[task_idx]]
    
    for msg in trajectory['trajectory']:
        if 'role' in msg and msg['role'] == 'user':
            print("========================================== [ 👤 User ] ============================================")
            print(f"{msg['content']}")
        elif 'role' in msg and msg['role'] == 'assistant':
            print("======================================== [ 🤖 Assistant ] ==========================================")
            print(f"{msg['content']}")
        elif 'type' in msg and msg['type'] == 'function_call':
            print("======================================== [ 🤖 Assistant ] ==========================================")
            print(f"{json.loads(msg['arguments'])['code']}")
        elif 'type' in msg and msg['type'] == 'function_call_output':
            print("================================== [ 🌏 Environment Feedback ] ====================================")
            print(f"{msg['output']}")
        print("====================================================================================================\n\n\n")

========================================== [ 👤 User ] ============================================
Using these 'APIs' and 'reflection history' of your previous actions, now generate code to solve the actual task:

My name is Edwin Wilson. 
My personal email is ed_wilson@gmail.com and phone number is 8506051563.

**Task**:
Give me a comma-separated list of top 3 most played indie song titles from across my Spotify song, album and playlist libraries.

**Reflection History**:
<reflection>
The Actor Agent's approach to solving the task had several issues:

1. **Missing API Calls**: The agent did not fetch play count data, which is crucial for determining the "most played" songs. It assumed the order of songs in the list represented play count, which is incorrect.

2. **Data Filtering**: The agent correctly filtered songs by the "indie" genre from the album library but failed to ensure that songs from the playlist library were also filtered by genre.

3. **Pagination**: The agent correctly 